Details of the solution

https://www.kaggle.com/competitions/birdclef-2022/discussion/327193

In [ ]:
#set up
!cp -r ../input/timm-pytorch-image-models /kaggle/working/
!pip install /kaggle/working/timm-pytorch-image-models/pytorch-image-models-master/
!mkdir /kaggle/working/logs
!mkdir output

In [ ]:
!pip install ../input/torchlibrosa/torchlibrosa-0.0.9-py2.py3-none-any.whl
!pip install ../input/birdclef-2022-wheels/timm-0.4.12-py3-none-any.whl

# CNN part

In [ ]:
## https://stackoverflow.com/questions/1868714/how-do-i-copy-an-entire-directory-of-files-into-an-existing-directory-using-pyth
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [ ]:
def transform(array):
    array = np.array(array)
    num_classes = 21
    num_chunks = 12
    
    predictions_per_file = num_classes * num_chunks
    array = array.reshape(-1, predictions_per_file)
    num_files, _ = array.shape
    new_array = np.empty((num_files, num_chunks, num_classes), dtype=array.dtype)
    for i, prediction in enumerate(array):
        new_array[i] = array[i].reshape(-1, num_classes).T.reshape(num_classes, num_chunks).T
    return new_array

def roll_with_pad(data, shift=0):
    axis = 0
    
    abs_shift = abs(shift)
    data = np.pad(data, [(abs_shift, abs_shift), (0, 0)])
    return data if shift == 0 else np.roll(data, shift=shift, axis=axis)[abs_shift:-abs_shift]

def postprocess_probabilities(probs):
    '''
        input: :param: probs - np.array of shape (num_files, num_chunks, num_classes)
        output: probs - np.array, postprocessed probabilities
    '''
    shifts = [-2, -1, 0, 1, 2]
    weights = np.array([0.5, 1.0, 7.0, 1.0, 0.5])
#     shifts = [-3, -2, -1, 0, 1, 2, 3]
#     weights = np.array([0.5, 1.0, 2.0, 7.0, 2.0, 1.0, 0.5])
    high_pass = 0.5
    
    num_files, num_classes, num_chunks = probs.shape
    
    for i, file_prediction in enumerate(probs):
        ## time-smoothing
        weighted_preds = []
        for shift, weight in zip(shifts, weights):
            weighted_preds.append(roll_with_pad(file_prediction, shift=shift) * weight / weights.sum())
        probs[i] = np.sum(weighted_preds, axis=0)
        
        #Vlomme's PP OFF
#         # https://github.com/vlomme/Birdcall-Identification-competition/blob/master/train.py postprocessing
#         max_probs = np.amax(probs[i], axis=0)
#         high_pass_mask = max_probs > high_pass
#         max_probs = np.where(high_pass_mask, max_probs, 0)
#         probs[i] += max_probs / 70
    
    return probs

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import librosa
import shutil
import subprocess
import pandas as pd
from os.path import join

# First, load list of audio files. We could use 'test.csv' as well,
# but for now, let's stick with parsing the test_soundscape folder.
test_audio_dir = '../input/birdclef-2022/test_soundscapes/'
scored_birds = ["akiapo", "aniani", "apapan", "barpet", "crehon", "elepai", "ercfra", \
                "hawama", "hawcre", "hawgoo", "hawhaw", "hawpet1", "houfin", "iiwi", "jabwar", "maupar", \
                "omao", "puaioh", "skylar", "warwhe1", "yefcan"]

## set high thresholds to birds which were predicted just fine on local validation
## is needed also to create per-model predictions
default_threshold = 0.04
thresholds = {k:v for k, v in zip(scored_birds, np.ones_like(scored_birds).astype(np.float32) * default_threshold)}
str_thresholds = ' '.join([f'{x:.4f}' for x in thresholds.values()])

thresholds["skylar"] = 0.5
# thresholds["elepai"] = 0.2
# thresholds["houfin"] = 0.2
# thresholds["jabwar"] = 0.4
# thresholds["yefcan"] = 0.3
# thresholds["maupar"] = 0.01

num_workers = 2
batch_size = 32

In [ ]:
## best ensemble
# experiment_names = ['exp-11-serv', 'exp6data', 'exp7serv', 'exp7v3', 'full-exp-5', 'start-fold4-exp10-full', 'start-fold4-exp13-full', 'start-fold4-exp15-full', 'start-fold4-exp17-full', 'new-metrics-exp-9-v2', 'new-metrics-exp-17', 'new-metrics-exp-18', 'new-metrics-exp-19']
# dirs_names = ['exp-11-serv', 'exp-6-serv', 'exp-7-serv', 'exp7', 'full_exp5', 'start_fold4_exp10_full', 'start_fold4_exp13_full_r', 'start_fold4_exp15_full', 'start_fold4_exp17_full', 'new-metrics-exp-9', 'new-metrics-exp-17', 'new-metrics-exp-18', 'new-metrics-exp-19']
# weights = np.array([1.0, 1.0, 1.0, 5.0, 1.0, 6.0, 7.0, 1.0, 2.0, 7.0, 8.0, 7.0, 7.0])
# folds = [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4]

## remove models w/o augs, must improve the score due to reduction of FP:
experiment_names = ['start-fold4-exp13-full', 'start-fold4-exp15-full', 'start-fold4-exp17-full', 'new-metrics-exp-9-v2', 'new-metrics-exp-17', 'new-metrics-exp-18', 'new-metrics-exp-19']
dirs_names = ['start_fold4_exp13_full_r', 'start_fold4_exp15_full', 'start_fold4_exp17_full', 'new-metrics-exp-9', 'new-metrics-exp-17', 'new-metrics-exp-18', 'new-metrics-exp-19']
weights = [7.0, 1.0, 2.0, 7.0, 8.0, 7.0, 7.0]
folds = [0, 0, 0, 4, 4, 4, 4]
configs = ['base_config.yaml'] * 7

## 100-epochs 5-fold ensemble effnet_v2s model
experiment_names.extend(['new-metrics-exp-23'] * 1)
dirs_names.extend(['new-metrics-exp-23'] * 1)
weights.extend([12.0] * 1)
folds.extend([4])
configs.extend(['tf_effnet_v2_s_in21k.yaml'] * 1)

weights = np.array(weights)
inference_on_dir_name = 'inferenceondirfastv16'

In [ ]:
for experiment_name in experiment_names:
    try:
        copytree(f'../input/{experiment_name}', './experiments/')
    except:
        print("Workadround about copy-tree file-exists behaviour")
shutil.copyfile(f'../input/{inference_on_dir_name}/inference_on_dir_different_thresholds.py', 'inference_on_dir.py')

for experiment_name, fold, config in zip(dirs_names, folds, configs):
    print(f"Running inference for experiment name : {experiment_name}")
    subprocess.run(f'python3 inference_on_dir.py --experiment-dir ./experiments \
                            --exp {experiment_name} \
                            --inference-dir ../input/birdclef-2022/test_soundscapes/ \
                            --thresholds {str_thresholds} \
                            --num-workers {num_workers} \
                            --batch-size {batch_size} \
                            --fold {fold} \
                            --cfg {config}', shell=True)

In [ ]:
subs_with_probs = []
for exp_dir in dirs_names:
    subs_with_probs.append(pd.read_csv(join('./experiments', exp_dir, 'submission_w_probs.csv')))

In [ ]:
for exp_name, sub in zip(experiment_names, subs_with_probs):
    print("{} : {:.4f}".format(exp_name, sub['score'].mean()))

In [ ]:
import shutil
shutil.rmtree('./experiments')

probs     = np.array([df['score'] for df in subs_with_probs])
new_probs = (probs * weights.reshape(-1, 1)).sum(axis=0) / weights.sum()

#transform
transformed_rows = transform(subs_with_probs[0].row_id)
transformed_probs = transform(new_probs)
transformed_rows.shape, transformed_probs.shape, transformed_rows[0, :3, :3]

#PP
print(new_probs.shape)
new_probs = postprocess_probabilities(transformed_probs.copy())
new_probs.mean(), new_probs.shape, new_probs[0, :, 0]

#sub
new_sub = pd.DataFrame({
    'row_id' : transformed_rows.flatten(),
    'target' : -2,
    'probability' : new_probs.flatten()
})
new_sub['bird'] = new_sub['row_id'].apply(lambda x : x.split('_')[2])

for bird, threshold in thresholds.items():
    print(f"{bird} : {threshold:.2f}")
    mask = new_sub.bird == bird
    new_sub.loc[mask, 'target'] = new_sub.loc[mask, 'probability'] > threshold
    
new_sub_with_proba = new_sub.copy()
new_sub = new_sub.drop(columns=['bird', 'probability'])
# new_sub.to_csv('submission.csv', index=False)
new_sub.head()

In [ ]:
#まとめ
df_ens_Slime   = new_sub_with_proba.copy()
df_ens_Slime.target.value_counts()

# SED part

In [ ]:
import pandas as pd
import numpy as np
import gc
import pickle
import yaml
from pathlib import Path

In [ ]:
# ==Ens==
def ens_weightave(df_ens1,df_ens2,
                  dict_thresh_short_ens1,dict_thresh_short_ens2,
                  dict_thresh_long_ens1,dict_thresh_long_ens2,
                  weight1,weight2):
    df_ens1 = df_ens1.rename(columns={'target':'target1','score':'score1','score_long':'score_long1'})
    df_ens2 = df_ens2.rename(columns={'target':'target2','score':'score2','score_long':'score_long2'})
    df_ens  = df_ens2.merge(df_ens1,on='row_id')
    row_id              = df_ens['row_id'].values
    probas_ens1         = df_ens['score1'].values
    probas_long_ens1    = df_ens['score_long1'].values
    probas_ens2         = df_ens['score2'].values
    probas_long_ens2    = df_ens['score_long2'].values
    #proba
    probas_ens        = weight1*probas_ens1       + weight2*probas_ens2
    probas_long_ens   = weight1*probas_long_ens1  + weight2*probas_long_ens2
    #thresh
    dict_thresh_short_ens={}
    dict_thresh_long_ens ={}
    for bird_name in dict_thresh_short_ens1.keys():
        thresh_ens1      = dict_thresh_short_ens1[bird_name]
        thresh_long_ens1 = dict_thresh_long_ens1[bird_name]
        thresh_ens2      = dict_thresh_short_ens2[bird_name]
        thresh_long_ens2 = dict_thresh_long_ens2[bird_name]
        thresh_ens       = weight1*thresh_ens1     +weight2*thresh_ens2
        thresh_long_ens  = weight1*thresh_long_ens1+weight2*thresh_long_ens2
        dict_thresh_short_ens.update({bird_name:thresh_ens})
        dict_thresh_long_ens.update({bird_name:thresh_long_ens})
    #get_sub
    pred        = {'row_id' : [], 'target' : [], 'score' : [], 'score_long' : []}
    row_ids     = list(row_id)
    for idx,row_id in enumerate(row_ids):
        soundscape_name = row_id.split('_')[0]+'_'+row_id.split('_')[1]
        bird_name       = row_id.split('_')[2]
        time_name       = row_id.split('_')[3]
        #proba
        proba_ens       = probas_ens[idx]
        proba_long_ens  = probas_long_ens[idx]
        #thresh
        thresh_ens      = dict_thresh_short_ens[bird_name]
        thresh_long_ens = dict_thresh_long_ens[bird_name]
        #judge
        event           = ((proba_ens >= thresh_ens).astype(int) \
                          + (proba_long_ens >= thresh_long_ens).astype(int)) >= 2#long,short
        pred['row_id'].append(row_id)
        pred['target'].append(event)
        pred['score'].append(proba_ens)
        pred['score_long'].append(proba_long_ens)
    #df
    df_ens          = pd.DataFrame(pred, columns=['row_id', 'target', 'score', 'score_long'])
    df_sub_ens      = df_ens[['row_id','target']]
    return df_ens,df_sub_ens,dict_thresh_short_ens,dict_thresh_long_ens

# ==PP==
def Slime_PP(df_ens,dict_thresh_short_ens,dict_thresh_long_ens):
    row_id          = df_ens['row_id'].values
    proba_ens       = df_ens['score'].values
    proba_long_ens  = df_ens['score_long'].values
    #
    transformed_row_id         = transform(row_id)
    transformed_proba_ens      = transform(proba_ens)
    transformed_proba_long_ens = transform(proba_long_ens)
    #pp
    pp_proba_ens       = postprocess_probabilities(transformed_proba_ens.copy())
    pp_proba_long_ens  = postprocess_probabilities(transformed_proba_long_ens.copy())
    #flatten
    pp_row_id          = transformed_row_id.flatten()
    pp_proba_ens       = pp_proba_ens.flatten()
    pp_proba_long_ens  = pp_proba_long_ens.flatten()
    #get_sub
    pred        = {'row_id' : [], 'target' : [], 'score' : [], 'score_long' : []}
    row_ids     = list(pp_row_id)
    for idx,row_id in enumerate(row_ids):
        soundscape_name = row_id.split('_')[0]+'_'+row_id.split('_')[1]
        bird_name       = row_id.split('_')[2]
        time_name       = row_id.split('_')[3]
        #thresh
        thresh_ens      = dict_thresh_short_ens[bird_name]
        thresh_long_ens = dict_thresh_long_ens[bird_name]
        #proba
        proba_ens       = pp_proba_ens[idx]
        proba_long_ens  = pp_proba_long_ens[idx]
        #judge
    #         event           = ((proba_ens >= thresh_ens).astype(int) \
    #                           + (proba_long_ens >= thresh_long_ens).astype(int)) >= 2#long and short
        event          = (proba_ens >= thresh_ens).astype(int) >= 1#short only
        pred['row_id'].append(row_id)
        pred['target'].append(event)
        pred['score'].append(proba_ens)
        pred['score_long'].append(proba_long_ens)

    #df
    df_pp_ens          = pd.DataFrame(pred, columns=['row_id', 'target', 'score', 'score_long'])
    df_sub_pp_ens      = df_pp_ens[['row_id','target']]
    return df_pp_ens,df_sub_pp_ens

SED Frontend1

In [ ]:
# ==================================================================================================================================
# Frontend1  = config_audio_inference_mel1
# ==========================Part1_1 : Frontend1 BCE Loss==========================#
models_path_nonfocal1  = {
                'exp178_fold0':'../input/bird3-infrence-data/exp178_seres26_exp124base_ep35_StepLR_seed1701_1fold_labelsmoothOFF_BCE2WayLoss/model/fold0/best.pt',
                'exp179_fold0':'../input/bird3-infrence-data/exp179_res34_exp124base_ep35_StepLR_seed1703_1fold_labelsmoothOFF_BCE2WayLoss/model/fold0/best.pt',
                'exp180_fold0':'../input/bird3-infrence-data/exp180_res50_exp124base_ep35_StepLR_seed1706_1fold_labelsmoothOFF_BCE2WayLoss/model/fold0/best.pt',
                'exp181_fold0':'../input/bird3-infrence-data/exp181_effv2s_exp124base_ep35_StepLR_seed1709_1fold_labelsmoothOFF_BCE2WayLoss/model/fold0/best.pt',
                }
configs_path_nonfocal1 = {
                 'exp178_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                 'exp179_fold0':'../input/bird3-infrence-data/exp142_resnet34_exp124base_seed115/yaml/config_audio.yaml',
                 'exp180_fold0':'../input/bird3-infrence-data/exp145_resnest50_exp124base_seed333/yaml/config_audio.yaml',
                 'exp181_fold0':'../input/bird3-infrence-data/exp143_effv2s_exp124base_seed55/yaml/config_audio2.yaml',
                }
ens_weights_nonfocal1  = {
                 'exp178_fold0':1,
                 'exp179_fold0':1,
                 'exp180_fold0':1,
                 'exp181_fold0':1,
                }
#==========================Part1_2 : Frontend1 Focalloss==========================#
models_path_focal1  = {
                    'exp124_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/model/fold0/best.pt',
                    'exp124_fold1':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/model/fold1/best.pt',
                    'exp124_fold2':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/model/fold2/best.pt',
                    'exp124_fold3':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/model/fold3/best.pt',
                    'exp148_fold0':'../input/bird3-infrence-data/exp148_seres26_exp124base_seed1411_1fold_ep35_StepLR/model/fold0/best.pt',
                    'exp153_fold0':'../input/bird3-infrence-data/exp153_effv2s_exp124base_seed4515_1fold_ep35_StepLR/model/fold0/best.pt',
                    'exp155_fold0':'../input/bird3-infrence-data/exp155_res50_exp124base_seed1995_1fold_ep35_StepLR/model/fold0/best.pt',
                    'exp159_fold0':'../input/bird3-infrence-data/exp159_res34_exp124base_seed4655_1fold_ep35_StepLR/model/fold0/best.pt',
                    }
configs_path_focal1 = { 
                     'exp124_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                     'exp124_fold1':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                     'exp124_fold2':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                     'exp124_fold3':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                     'exp148_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                     'exp153_fold0':'../input/bird3-infrence-data/exp143_effv2s_exp124base_seed55/yaml/config_audio2.yaml',
                     'exp155_fold0':'../input/bird3-infrence-data/exp145_resnest50_exp124base_seed333/yaml/config_audio.yaml',
                     'exp159_fold0':'../input/bird3-infrence-data/exp142_resnet34_exp124base_seed115/yaml/config_audio.yaml',
                   }
ens_weights_focal1  = { 
                     'exp124_fold0':1,
                     'exp124_fold1':1,
                     'exp124_fold2':1,
                     'exp124_fold3':1,
                     'exp148_fold0':1,
                     'exp153_fold0':1,
                     'exp155_fold0':1,
                     'exp159_fold0':1,
                    }
#==========================save==========================#
models_path              = dict()
models_path['focal']     = models_path_focal1
models_path['nonfocal']  = models_path_nonfocal1

configs_path             = dict()
configs_path['focal']    = configs_path_focal1
configs_path['nonfocal'] = configs_path_nonfocal1

ens_weights              = dict()
ens_weights['focal']     = ens_weights_focal1
ens_weights['nonfocal']  = ens_weights_nonfocal1

with open('./output/models_path.pickle', mode='wb') as f:
    pickle.dump(models_path,f)
with open('./output/configs_path.pickle', mode='wb') as f:
    pickle.dump(configs_path,f)
with open('./output/ens_weights.pickle', mode='wb') as f:
    pickle.dump(ens_weights,f)
    
#==========================inference==========================#
!python ../input/bird3-infrence-data-ens/bird3_ens/train_cl_birds3_inference_fast_v3.py --config 'config_audio_inference_mel1' --num-workers 4 --batch-size 32  --coeff-thresh 1.0 --mode-ens Average --testmode OFF

#==========================BCE Loss==========================#
df_ens1_nonfocal              = pd.read_csv('./output/sub_ens_nonfocal.csv')
df_sub_ens1_nonfocal          = df_ens1_nonfocal[['row_id','target']]
df_sub_ens1_nonfocal.to_csv("submission.csv", index=False)
#thresh
with open('./output/dict_thresh_short_ens_nonfocal.pickle', 'rb') as pickle_file:
    dict_thresh_short_ens1_nonfocal  = pickle.load(pickle_file)
with open('./output/dict_thresh_long_ens_nonfocal.pickle', 'rb') as pickle_file:
    dict_thresh_long_ens1_nonfocal   = pickle.load(pickle_file)

print('-------------BCE Loss------------------------')
print('pred_mean      :',np.round(df_ens1_nonfocal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens1_nonfocal['score_long'].values.mean(),5))  
print('--------------------------------------------------')


#==========================Focal Loss==========================#
df_ens1_focal                 = pd.read_csv('./output/sub_ens_focal.csv')
df_sub_ens1_focal             = df_ens1_focal[['row_id','target']]
df_sub_ens1_focal.to_csv("submission.csv", index=False)
#thresh
with open('./output/dict_thresh_short_ens_focal.pickle', 'rb') as pickle_file:
    dict_thresh_short_ens1_focal  = pickle.load(pickle_file)
with open('./output/dict_thresh_long_ens_focal.pickle', 'rb') as pickle_file:
    dict_thresh_long_ens1_focal   = pickle.load(pickle_file)

print('-------------Focal Loss------------------------')
print('pred_mean      :',np.round(df_ens1_focal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens1_focal['score_long'].values.mean(),5))  
print('--------------------------------------------------')

gc.collect()
    

SED Frontend2

In [ ]:
# ====================================================================================#
# Frontend2  = config_audio_inference_mel3  
# ==========================Part2_1 : Frontend2 BCE LOSS==========================#
models_path_nonfocal2  = {
                    'exp185_fold0':'../input/bird3-infrence-data/exp185_seres26_exp124base_seed2016_1fold_labelsmoothOFF_BCE2WayLoss_chunk15_changemel/model/fold0/best.pt', 
                    'exp186_fold0':'../input/bird3-infrence-data/exp186_effv2s_exp124base_seed2031_1fold_labelsmoothOFF_BCE2WayLoss_chunk15_changemel/model/fold0/best.pt', 
                    'exp187_fold0':'../input/bird3-infrence-data/exp187_res34_exp124base_seed2032_1fold_labelsmoothOFF_BCE2WayLoss_chunk15_changemel/model/fold0/best.pt', 
                    'exp188_fold0':'../input/bird3-infrence-data/exp188_res50_exp124base_seed2034_1fold_labelsmoothOFF_BCE2WayLoss_chunk15_changemel/model/fold0/best.pt', 
                }

configs_path_nonfocal2 = {
                    'exp185_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml', 
                    'exp186_fold0':'../input/bird3-infrence-data/exp143_effv2s_exp124base_seed55/yaml/config_audio2.yaml',    
                    'exp187_fold0':'../input/bird3-infrence-data/exp142_resnet34_exp124base_seed115/yaml/config_audio.yaml',  
                    'exp188_fold0':'../input/bird3-infrence-data/exp145_resnest50_exp124base_seed333/yaml/config_audio.yaml', 
                }

ens_weights_nonfocal2  = {
                     'exp185_fold0':1, 
                     'exp186_fold0':1, 
                     'exp187_fold0':1, 
                     'exp188_fold0':1, 
                }

#==========================Part1_2 : Frontend1 Focalloss==========================#
models_path_focal2  = {
                    'exp166_fold0':'../input/bird3-infrence-data/exp166_effv2s_exp124base_seed1341_1fold_ep35_StepLR_chunk15_changemel/model/fold0/best.pt',
                    'exp167_fold0':'../input/bird3-infrence-data/exp167_seres26_exp124base_seed1351_1fold_ep35_StepLR_chunk15_changemel/model/fold0/best.pt',
                    'exp170_fold0':'../input/bird3-infrence-data/exp170_res50_exp124base_seed1451_1fold_ep35_StepLR_chunk15_changemel/model/fold0/best.pt',
                    'exp171_fold0':'../input/bird3-infrence-data/exp171_res34_exp124base_seed1461_1fold_ep35_StepLR_chunk15_changemel/model/fold0/best.pt',
                }

configs_path_focal2 = { 
                    'exp166_fold0':'../input/bird3-infrence-data/exp143_effv2s_exp124base_seed55/yaml/config_audio2.yaml',
                    'exp167_fold0':'../input/bird3-infrence-data/exp124_seres26_chunk10_pseudoexp044_strong2_ep30_cutmix05_pseudo2ndmaskON_randlowpassOFF/yaml/config_audio4.yaml',
                    'exp170_fold0':'../input/bird3-infrence-data/exp145_resnest50_exp124base_seed333/yaml/config_audio.yaml',
                    'exp171_fold0':'../input/bird3-infrence-data/exp142_resnet34_exp124base_seed115/yaml/config_audio.yaml',
               }

ens_weights_focal2  = { 
                     'exp166_fold0':1,
                     'exp167_fold0':1,
                     'exp170_fold0':1,
                     'exp171_fold0':1,
                }

#==========================合体==========================#
models_path              = dict()
models_path['focal']     = models_path_focal2
models_path['nonfocal']  = models_path_nonfocal2

configs_path             = dict()
configs_path['focal']    = configs_path_focal2
configs_path['nonfocal'] = configs_path_nonfocal2

ens_weights              = dict()
ens_weights['focal']     = ens_weights_focal2
ens_weights['nonfocal']  = ens_weights_nonfocal2

with open('./output/models_path.pickle', mode='wb') as f:
    pickle.dump(models_path,f)
with open('./output/configs_path.pickle', mode='wb') as f:
    pickle.dump(configs_path,f)
with open('./output/ens_weights.pickle', mode='wb') as f:
    pickle.dump(ens_weights,f)

#==========================inference==========================#
!python ../input/bird3-infrence-data-ens/bird3_ens/train_cl_birds3_inference_fast_v3.py --config 'config_audio_inference_mel3' --num-workers 4 --batch-size 32  --coeff-thresh 1.0 --mode-ens Average --testmode OFF

#==========================BCE LOSS==========================#
df_ens2_nonfocal              = pd.read_csv('./output/sub_ens_nonfocal.csv')
df_sub_ens2_nonfocal          = df_ens2_nonfocal[['row_id','target']]
df_sub_ens2_nonfocal.to_csv("submission.csv", index=False)

#thresh
with open('./output/dict_thresh_short_ens_nonfocal.pickle', 'rb') as pickle_file:
    dict_thresh_short_ens2_nonfocal  = pickle.load(pickle_file)
with open('./output/dict_thresh_long_ens_nonfocal.pickle', 'rb') as pickle_file:
    dict_thresh_long_ens2_nonfocal   = pickle.load(pickle_file)

print('-------------BCE LOSS------------------------')
print('pred_mean      :',np.round(df_ens2_nonfocal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens2_nonfocal['score_long'].values.mean(),5))  
print('--------------------------------------------------')

#==========================Focal LOSS==========================#
df_ens2_focal                 = pd.read_csv('./output/sub_ens_focal.csv')
df_sub_ens2_focal             = df_ens2_focal[['row_id','target']]
df_sub_ens2_focal.to_csv("submission.csv", index=False)

#thresh
with open('./output/dict_thresh_short_ens_focal.pickle', 'rb') as pickle_file:
    dict_thresh_short_ens2_focal  = pickle.load(pickle_file)
with open('./output/dict_thresh_long_ens_focal.pickle', 'rb') as pickle_file:
    dict_thresh_long_ens2_focal   = pickle.load(pickle_file)

print('-------------Focal LOSS------------------------')
print('pred_mean      :',np.round(df_ens2_focal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens2_focal['score_long'].values.mean(),5))  
print('--------------------------------------------------')

gc.collect()
    

SED ensemble

In [ ]:
#BCE LOSS
weight_nonfocal_mel1=0.7
weight_nonfocal_mel2=0.3
df_ens_nonfocal,df_sub_ens_nonfocal,dict_thresh_short_ens_nonfocal,dict_thresh_long_ens_nonfocal = ens_weightave(df_ens1_nonfocal,df_ens2_nonfocal,
                                                                                                      dict_thresh_short_ens1_nonfocal,dict_thresh_short_ens2_nonfocal,
                                                                                                      dict_thresh_long_ens1_nonfocal,dict_thresh_long_ens2_nonfocal,
                                                                                                      weight1=weight_nonfocal_mel1,weight2=weight_nonfocal_mel2)
#Focal LOSS
weight_focal_mel1=0.7
weight_focal_mel2=0.3
df_ens_focal,df_sub_ens_focal,dict_thresh_short_ens_focal,dict_thresh_long_ens_focal =  ens_weightave(df_ens1_focal,df_ens2_focal,
                                                                                                      dict_thresh_short_ens1_focal,dict_thresh_short_ens2_focal,
                                                                                                      dict_thresh_long_ens1_focal,dict_thresh_long_ens2_focal,
                                                                                                      weight1=weight_focal_mel1,weight2=weight_focal_mel2)

print('------------------------BCE LOSS-------------------------------')
print('pred_mean      :',np.round(df_ens_nonfocal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens_nonfocal['score_long'].values.mean(),5))   
print('-------------------------------------------------------------------')

print('------------------------Focal LOSS-------------------------------')
print('pred_mean      :',np.round(df_ens_focal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens_focal['score_long'].values.mean(),5))   
print('-------------------------------------------------------------------')

SED PP for BCEloss

In [ ]:
df_ens_nonfocal,df_sub_ens_nonfocal = Slime_PP(df_ens_nonfocal,dict_thresh_short_ens_nonfocal,dict_thresh_long_ens_nonfocal)

print('------------------------BCE LOSS Afte PP-------------------------------')
print('pred_mean      :',np.round(df_ens_nonfocal['score'].values.mean(),5))    
print('pred_long_mean :',np.round(df_ens_nonfocal['score_long'].values.mean(),5))   
print('-------------------------------------------------------------------')


SED Output

In [ ]:
df_ens_UEMU_NonFocal = df_ens_nonfocal.copy()
df_ens_UEMU_Focal    = df_ens_focal.copy()

# Team Ensembling part

Group1:14Birds (CNN + SED ,BCE loss)

In [ ]:
# =======Params============================
#thresh
default_threshold              = 0.05 #0.05
thresholds_ens                 = {k:v for k, v in zip(scored_birds, np.ones_like(scored_birds).astype(np.float32) * default_threshold)}
thresholds_ens["skylar"]       = 0.35

#weight
weight_Slime                   = 0.35
weight_UEMU                    = 0.65

# =========================================
#Ensemble
df_ens_UEMU_NonFocal           = df_ens_UEMU_NonFocal.sort_values(['row_id'])
df_ens_Slime                   = df_ens_Slime.sort_values(['row_id'])

df_ens_NonFocal                = df_ens_Slime.copy()
df_ens_NonFocal['probability'] = weight_Slime*df_ens_Slime['probability'] + weight_UEMU*df_ens_UEMU_NonFocal['score'] 

for bird, threshold in thresholds_ens.items():
    print(f"{bird} : {threshold:.2f}")
    mask = df_ens_NonFocal.bird == bird
    df_ens_NonFocal.loc[mask, 'target'] = df_ens_NonFocal.loc[mask, 'probability'] > threshold

#sub
df_ens_sub = df_ens_NonFocal.drop(columns=['probability', 'bird'])
df_ens_sub.to_csv('submission.csv', index=False)

Group2:7Birds (SED ,Focal loss)

In [ ]:
# =======Params============================
#Select bird
bird_Focal      = ['crehon','ercfra','hawgoo','hawhaw','hawpet1','maupar','puaioh']
#     bird_Focal      = ['crehon','ercfra','hawama','hawcre','hawgoo','hawhaw','hawpet1','maupar','puaioh']

# =========================================
df_ens_Focal    = df_ens_UEMU_Focal.copy()

bird_NonFocal   = list(set(scored_birds)-set(bird_Focal))
print('bird_Focal',len(bird_Focal),bird_Focal)
print('bird_NonFocal',len(bird_NonFocal),bird_NonFocal)

df_ens_NonFocal = df_ens_NonFocal.rename(columns={'target':'target_NonFocal'})
df_ens_Focal    = df_ens_Focal.rename(columns={'target':'target_Focal'})
df_ens_Team     = df_ens_NonFocal[['row_id','target_NonFocal','bird']].copy()
df_ens_Team     = df_ens_Team.merge(df_ens_Focal[['row_id','target_Focal']],on='row_id')

targets    = []
for idx in range(len(df_ens_Team)):
    if df_ens_Team['bird'][idx] in bird_Focal:
        targets.append(df_ens_Team['target_Focal'][idx])
    else:
        targets.append(df_ens_Team['target_NonFocal'][idx])
df_ens_Team['target'] = targets

#sub
df_ens_sub = df_ens_Team.drop(columns=['target_NonFocal','target_Focal', 'bird'])
df_ens_sub.to_csv('submission.csv', index=False)

In [ ]:
df_ens_Team['target'].value_counts()

Hist BCE loss Part

In [ ]:
df_ens_Slime['probability'].hist(bins=100)

In [ ]:
df_ens_UEMU_NonFocal['score'].hist(bins=100)

Hist Focal loss Part

In [ ]:
df_ens_Focal['score'].hist(bins=100)

In [ ]:
df_ens_Focal['score_long'].hist(bins=100)

In [ ]:
# df_ens_Team['probability'].hist(bins=100)